# IS20140 Final Project - Overview

## The Idea:

To create a version of Mat2Vec trained on data from the social sciences as opposed to data from the material sciences.

## The Implementation: 

NB Need to grab the citation for T-5

## Install all required files

In [1]:
#Import modules required to run 8-bit T-5 network
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate
!pip install --quiet sentencepiece
!pip install --quiet tokenizers
!pip install --quiet datasets
!pip install --quiet evaluate
!pip install --quiet torch
!pip install --quiet numpy
!pip install --quiet pandas
!pip install --quiet sentence-transformers
!pip install --quiet sklearn

     |████████████████████████████████| 62.5 MB 1.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 27.9 MB/s 
     |████████████████████████████████| 163 kB 56.5 MB/s 
     |████████████████████████████████| 148 kB 35.6 MB/s 
     |████████████████████████████████| 1.3 MB 32.0 MB/s 
     |████████████████████████████████| 441 kB 36.9 MB/s 
     |████████████████████████████████| 212 kB 70.8 MB/s 
     |████████████████████████████████| 115 kB 70.4 MB/s 
     |████████████████████████████████| 95 kB 5.2 MB/s 
     |████████████████████████████████| 127 kB 66.8 MB/s 
     |████████████████████████████████| 115 kB 75.8 MB/s 
     |████████████████████████████████| 72 kB 1.5 MB/s 
     |████████████████████████████████| 85 kB 3.3 MB/s 


## Preprocess actual data

In [2]:
#Import and preprocess dataset
import pandas as pd
dataset = pd.read_excel("/content/drive/MyDrive/College/IS20140/Final Project (1)/savedrecs-tagSociology_HighlyCited-date26092022-1.xlsx")
dataset = dataset.dropna(axis=0) #Drop rows with no known abstract
#print(dataset.head())
print(dataset)

                                         Article Title  \
0    For a Sociology of Expertise: The Social Origi...   
1                       Machine Learning for Sociology   
2    Bright Futures in Malawi's New Dawn: Education...   
4    Strategic unknowns: towards a sociology of ign...   
5    Toward a Comparative Sociology of Valuation an...   
..                                                 ...   
165  Pladias Database of the Czech Flora and Vegeta...   
166  Comparative International Entrepreneurship: A ...   
167  Barriers of Influenza Vaccination Intention an...   
168  The human dimensions of energy use in building...   
169  Characterising and justifying sample size suff...   

                                              Abstract  
0    This article endeavors to replace the sociolog...  
1    Machine learning is a field at the intersectio...  
2    Imagined futures, once a vital topic of theore...  
4    Developing an agenda for the social study of i...  
5    This review d

## Test and fine-tune model

Import AllenAI SPECTER Model (To get the similarity of the abstracts)

* Once we've done that we can get a topic model printed

References: 

    @inproceedings{specter2020cohan,
      title={{SPECTER: Document-level Representation Learning using Citation-informed Transformers}},
      author={Arman Cohan and Sergey Feldman and Iz Beltagy and Doug Downey and Daniel S. Weld},
    booktitle={ACL},
    year={2020}
    }

In [3]:
#Get cosine similarity between all abstracts. Use this in a topic model.
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

abstracts = dataset["Abstract"].to_numpy() #Convert to NumPy array, otherwise the encoder gets confused

model = SentenceTransformer('sentence-transformers/allenai-specter')
embeddings = model.encode(abstracts)
similarity = cosine_similarity([embeddings[0]], embeddings)
  #Interestingly enough, it doesn't REALLY matter which specific sentence you choose to run the similarity against, they will always cluster around each other.
    #I suppose the encodings are always the same, so that makes a certain amount of sense.

print(similarity) #Sorts in ascending order.
torch.cuda.empty_cache() #Clears the CUDA cache and frees up space in the GPU

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/462k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

[[0.99999994 0.7256185  0.68239653 0.8030982  0.7209832  0.7364234
  0.74341714 0.75400424 0.7703593  0.6403638  0.7670264  0.7372112
  0.7221168  0.7614993  0.7910791  0.7977913  0.74930674 0.6881014
  0.75378525 0.79095834 0.65777713 0.63305366 0.7705231  0.6365406
  0.7173687  0.7059687  0.6631863  0.776371   0.75239027 0.6973363
  0.4984904  0.7473954  0.75897896 0.71313846 0.77687454 0.7600129
  0.5899486  0.81174576 0.77325416 0.75563616 0.699718   0.7021886
  0.66475004 0.8081634  0.71391594 0.7660041  0.67994034 0.6608818
  0.74376893 0.72087276 0.7721344  0.78934145 0.6915561  0.79366857
  0.65753657 0.7090231  0.7171173  0.7694791  0.57134736 0.6775509
  0.69497603 0.6936115  0.7964656  0.64224195 0.7548892  0.7163773
  0.71531355 0.67829555 0.7833202  0.6589559  0.7388942  0.68706214
  0.7952227  0.7441417  0.68412226 0.6867175  0.7185894  0.7884297
  0.77249515 0.5319327  0.7073685  0.6379434  0.72920597 0.56772107
  0.66822624 0.7357922  0.6023432  0.6667559  0.7159269  0.

Import T5 For Question Generation Model (To produce sample questions which will be used to train the T5 model down below.

* This is one of the models we just *use* to get some slightly better training data. We don't need to fine-tune this one, we just use it as-is, being careful to focus on the Abstracts.

Reference: https://huggingface.co/iarfmoose/t5-base-question-generator

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("iarfmoose/t5-base-question-generator")
model = AutoModelForSeq2SeqLM.from_pretrained("iarfmoose/t5-base-question-generator")

all_abstracts = dataset["Abstract"].to_numpy()
max_new_tokens = 50

questions_list = list()

for abstract in all_abstracts:
    #Tokenize each abstract, get the question, and append it to the dataset.
    input_ids = tokenizer(abstract, return_tensors="pt").input_ids  
    outputs = model.generate(input_ids, max_new_tokens=max_new_tokens)
    resultant_question = tokenizer.decode(outputs[0], skip_special_tokens=True)
    questions_list.append(resultant_question)
    
dataset["RQ"] = questions_list

print(dataset)
dataset.to_csv("sample_dataset_with_rqs.csv")

torch.cuda.empty_cache() #Clear cache to make space for T5 we'll be training in a moment.

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/121 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

                                         Article Title  \
0    For a Sociology of Expertise: The Social Origi...   
1                       Machine Learning for Sociology   
2    Bright Futures in Malawi's New Dawn: Education...   
4    Strategic unknowns: towards a sociology of ign...   
5    Toward a Comparative Sociology of Valuation an...   
..                                                 ...   
165  Pladias Database of the Czech Flora and Vegeta...   
166  Comparative International Entrepreneurship: A ...   
167  Barriers of Influenza Vaccination Intention an...   
168  The human dimensions of energy use in building...   
169  Characterising and justifying sample size suff...   

                                              Abstract  \
0    This article endeavors to replace the sociolog...   
1    Machine learning is a field at the intersectio...   
2    Imagined futures, once a vital topic of theore...   
4    Developing an agenda for the social study of i...   
5    This rev

Import t-5-8-bit

### Use models as they are:

In [ ]:
#Tokenize dataset
import datasets as hf_datasets
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

model_name = "t5-3b-sharded" #@param ["t5-11b-sharded", "t5-3b-sharded"]
#Form dropdown menu automatically included, nice!
model_id=f"ybelkada/{model_name}"

tokenizer = T5Tokenizer.from_pretrained(model_id)
model_8bit = T5ForConditionalGeneration.from_pretrained(model_id, device_map="auto", load_in_8bit=True, return_dict=True)

def tokenize_abstracts(examples):
  return tokenizer(examples["Abstract"], padding=True, truncation=True)

def tokenize_titles(examples):
  return tokenizer(examples["Article Title"], padding=True, truncation=True)

def tokenize_research_questions(examples):
  return tokenizer(examples["RQ"], padding=True, truncation=True)

hf_dataset = hf_datasets.Dataset.from_pandas(dataset)
tokenized_abstracts = hf_dataset.map(tokenize_abstracts, batched=True)
tokenized_rqs = hf_dataset.map(tokenize_research_questions, batched=True)
#tokenized_abstracts = tokenized_abstracts.remove_columns("RQ", "Article Title")
tokenized_rqs = tokenized_rqs.remove_columns("Abstract")
tokenized_rqs = tokenized_rqs.remove_columns("Article Title")
tokenized_rqs = tokenized_rqs.rename_column("RQ", "Decoder RQ")
tokenized_rqs = tokenized_rqs.rename_column("input_ids", "decoder_input_ids")
tokenized_rqs = tokenized_rqs.rename_column('attention_mask', 'decoder_attention_mask')
tokenized_rqs = tokenized_rqs.rename_column('__index_level_0__', 'decoder__index_level_0__')

tokenized_dataset = hf_datasets.concatenate_datasets([tokenized_abstracts, tokenized_rqs], axis=1)
tokenized_dataset = tokenized_dataset.remove_columns(['decoder__index_level_0__', "Decoder RQ"])

tokenized_split_dataset = tokenized_dataset.train_test_split(test_size=0.33)

#print("Tokenized abstracts:", tokenized_abstracts)
#print("Tokenized titles:", tokenized_titles)
print("Tokenized dataset:", tokenized_split_dataset)

loading file spiece.model from cache at /root/.cache/huggingface/hub/models--ybelkada--t5-3b-sharded/snapshots/4b138fac625f7c009822fdee40780671d45325a4/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ybelkada--t5-3b-sharded/snapshots/4b138fac625f7c009822fdee40780671d45325a4/config.json
Model config T5Config {
  "_name_or_path": "ybelkada/t5-3b-sharded",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 16384,
  "d_kv": 128,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_he

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

Tokenized dataset: DatasetDict({
    train: Dataset({
        features: ['Article Title', 'Abstract', 'RQ', '__index_level_0__', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 113
    })
    test: Dataset({
        features: ['Article Title', 'Abstract', 'RQ', '__index_level_0__', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
        num_rows: 56
    })
})


### Use custom/fine-tuned model

Labels. The labels are an optional argument which can be passed in order for the model to compute the loss itself. These labels should be the expected prediction of the model: it will use the standard loss in order to compute the loss between its predictions and the expected value (the label).

In [ ]:
#Fine-tune 8-bit model
#from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
import numpy as np
import accelerate
import evaluate
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="test_trainer", 
    per_device_train_batch_size=1, 
    gradient_accumulation_steps=4, 
    gradient_checkpointing=True, 
    fp16=False, 
    optim="adafactor", 
    #label_names=["input_ids", "decoder_input_ids"],
    label_names=["decoder_input_ids", "decoder_attention_mask"],
) #Set output directory

metric = evaluate.load("accuracy") #Set metric to evaluate model performance

trainer = Trainer(
    model=model_8bit,
    args=training_args,
    train_dataset=tokenized_split_dataset["train"],
    eval_dataset=tokenized_split_dataset["test"],
    #compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer.train() #NB Looks like we need to drop the abstract and only give it the input IDs.
#trainer.save_model("test_trainer/saved_model")
#trainer.save_pretrained("test_trainer/saved_model")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: RQ, Abstract, Article Title, __index_level_0__. If RQ, Abstract, Article Title, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 113
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 84
  Number of trainable parameters = 33026048
`use_cache=True` is incompatible with grad

ValueError: ignored

In [ ]:
torch.cuda.empty_cache()

------

#### Scratchpad - debugging training issues

    ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values,encoder_last_hidden_state. For reference, the inputs it received are input_ids,attention_mask,decoder_input_ids,decoder_attention_mask.

If I had to guess, this is an issue with the compute metrics.

    metric = evaluate.load("accuracy") #Set metric to evaluate model performance

And

    return metric.compute(predictions=predictions, references=labels)



Compute Metrics are defined in the ```compute_metrics``` section of the ```Trainer``` class, as shown below:

    trainer = Trainer(
        model=model_8bit,
        args=training_args,
        train_dataset=tokenized_split_dataset["train"],
        eval_dataset=tokenized_split_dataset["test"],
        compute_metrics=compute_metrics,
    )

And the ```compute_metrics``` function is defined in 

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)



In [ ]:
# Scratchpad - Use Trainer without any custom compute metrics.
  # I remember this came from an issue with the inputs...

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
import numpy as np
import accelerate
import evaluate
import torch

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

selected_model = "t5-3b-sharded" #@param ["t5-11b-sharded", "t5-3b-sharded"]
model_id=f"ybelkada/{model_name}"

model_8bit = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

training_args = TrainingArguments(
    output_dir="test_trainer",  #Set output directory
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    optim="adafactor",
    label_names=["input_ids", "decoder_input_ids"],
    include_inputs_for_metrics=True,
    )
metric = evaluate.load("accuracy") #Set metric to evaluate model performance
  #NB That metric above is the loss function. MSE is mostly used for regression tasks.

trainer = Trainer(
    model=model_8bit,
    args=training_args,
    train_dataset=tokenized_split_dataset["train"],
    eval_dataset=tokenized_split_dataset["test"],
    compute_metrics=compute_metrics,
)

trainer.train() #NB Looks like we just need to figure out how to compute the loss.
  #It may be getting confused by the attention masks, since
#trainer.save_model("test_trainer/saved_model")
#trainer.save_pretrained("test_trainer/saved_model")

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Article Title, Abstract, __index_level_0__. If Article Title, Abstract, __index_level_0__ are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 113
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 84
  Number of trainable parameters = 33026048
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


ValueError: ignored

# Run saved Model

In [ ]:
#Use model
#We know this will break, that's okay.
new_model_8bit = "test_trainer/saved_model"
trained_tokenizer = AutoTokenizer.from_pretrained(new_model_8bit)
trained_model_8bit = AutoModelForSeq2SeqLM.from_pretrained(new_model_8bit, device_map="auto", load_in_8bit=True)

#Use the new 8-bit model:
max_new_tokens = 50
input_string = "What causes the relationship between metrics, markets and affect in the contemporary UK academy?" #Need to fill this with a relevant string. We know T5 can draw inferences and answer questions based on training data.

input_ids = trained_tokenizer(
    input_string, return_tensors="pt"
).input_ids  #Set this to an input parameter. Also need to make sure we're setting lengths appropriately.

outputs = trained_model_8bit.generate(input_ids, max_new_tokens=max_new_tokens)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

OSError: ignored

# **References (get these into the right citation style)**

https://huggingface.co/blog/hf-bitsandbytes-integration

https://discuss.huggingface.co/t/t5-finetuning-tips/684/5

https://colab.research.google.com/github/patil-suraj/exploring-T5/blob/master/T5_on_TPU.ipynb#scrollTo=QLGiFCDqvuil 